# Content Based Filtering

## Importación de librerías

In [54]:
import os
import re
import ast
import json
import nltk
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import linear_kernel
from sklearn.manifold import TSNE

### Descarga de las stopwords y el lematizador

Descargamos de *NLTK* las stopwords y *WordNet* para el lematizador.

In [55]:
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package wordnet to /Users/maria/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/maria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Carga de datos

Creamos una función para cargar los json, con posibilidad de cargar un % de los mismos.

In [ ]:
DIR = os.path.join(os.path.dirname(os.path.dirname(__file__)), 'Dataset')

def load_data(file_name, sample_fraction=1, seed=42):
    """
    Carga una fracción de los datos desde un archivo JSON.
    
    Parámetros:
    - file_name: Ruta del archivo.
    - sample_fraction: Fracción de datos a cargar (por defecto 100%).
    - seed: Semilla para asegurar reproducibilidad.
    
    Devuelve:
    - Un DataFrame con la muestra de datos.
    """
    random.seed(seed)  # Fijar semilla para reproducibilidad

    data = []
    with open(file_name, 'r', encoding='utf-8') as fin:
        for line in fin:
            if random.random() < sample_fraction:  # Probabilidad fija basada en la semilla
                data.append(json.loads(line))

    print(f"Cargados {len(data)} registros aproximadamente ({sample_fraction*100}% del total)")
    
    return pd.DataFrame(data)

Cargamos los json de `books`, con los metadatos de los libros; `authors`, con los datos de los autores; y `genres`, con los géneros por libro.

In [57]:
books = pd.read_json(os.path.join(DIR, 'goodreads_books_young_adult.json'), lines=True)
books.head()

,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,kindle_asin,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,,1,[147734],US,,"[{'count': '1057', 'name': 'to-read'}, {'count...",B0056A00P4,true,4.04,B0056A00P4,...,,,,https://www.goodreads.com/book/show/12182387-t...,https://s.gr-assets.com/assets/nophoto/book/11...,12182387,4,285263,"The Passion (Dark Visions, #3)","The Passion (Dark Visions, #3)"
1,,2,[425995],US,,"[{'count': '1010', 'name': 'to-read'}, {'count...",B006KLYIAG,true,3.80,B006KLYIAG,...,,,,https://www.goodreads.com/book/show/20135365-h...,https://s.gr-assets.com/assets/nophoto/book/11...,20135365,5,18450480,Hope's Daughter,Hope's Daughter
2,0698143760,17,[493993],US,,"[{'count': '1799', 'name': 'fantasy'}, {'count...",,true,3.80,,...,3,,2014,https://www.goodreads.com/book/show/21401181-h...,https://images.gr-assets.com/books/1394747643m...,21401181,33,24802827,"Half Bad (Half Life, #1)","Half Bad (Half Life, #1)"
3,,9,[176160],US,eng,"[{'count': '7173', 'name': 'to-read'}, {'count...",B0042JSOQC,true,4.35,B004IYJDXY,...,,,,https://www.goodreads.com/book/show/10099492-t...,https://s.gr-assets.com/assets/nophoto/book/11...,10099492,152,10800440,Twelfth Grade Kills (The Chronicles of Vladimi...,Twelfth Grade Kills (The Chronicles of Vladimi...
4,0990662616,428,[],US,eng,"[{'count': '9481', 'name': 'to-read'}, {'count...",,false,3.71,B00MW0MTGE,...,10,Special Edition,2014,https://www.goodreads.com/book/show/22642971-t...,https://images.gr-assets.com/books/1406979059m...,22642971,1525,42144295,The Body Electric,The Body Electric


In [59]:
authors = load_data(os.path.join(DIR, 'goodreads_book_authors.json.gz'))
authors.head()

Cargados 829529 registros aproximadamente (100% del total)


,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [60]:
genres = load_data(os.path.join(DIR, 'goodreads_book_genres_initial.json.gz'))
genres.head()

Cargados 2360655 registros aproximadamente (100% del total)


,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


## Preparación de datos

Definimos funciones para preprocesar el texto utilizando *regex*, para eliminar las stopwords y para usar el lematizador.

In [61]:
def clean_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\d+', '', text)  # Eliminar números
    text = re.sub(r'[^\w\s]', '', text)  # Eliminar puntuación
    text = text.strip()  # Eliminar espacios extra
    return text

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

Ahora, definimos funciones para realizar operaciones de ingeniería de características. Principalmente, lo hacemos sobre `books`y sobre la unión de los 3 datasets, `books_authors_genres`.

In [62]:
def preprocess_books(books_df):
    books_df = books_df[books_df['language_code'] == 'eng']
    books_df['author_id'] = books_df['authors'].apply(lambda x: [author['author_id'] for author in x] if x else None)
    books_df = books_df.explode('author_id').reset_index(drop=True)
    return books_df.drop(columns=['country_code', 'kindle_asin', 'link', 'authors', 'isbn13', 'url'])


def merge_books_authors_genres(books_df, authors_df, genres_df):
    books_authors = books_df.merge(authors_df, on='author_id', how='left', suffixes=('', '_author'))
    genres_df['book_id'] = genres_df['book_id'].astype(int)
    return books_authors.merge(genres_df, on='book_id', how='left', suffixes=('', '_genre'))


def clean_and_engineer_features(df):
    # Limpieza básica
    df['popular_shelves'] = df['popular_shelves'].apply(lambda x: [shelf['name'] for shelf in x] if x else [''])
    df['genres'] = df['genres'].apply(lambda x: list(x.keys()) if x else [''])
    df['series'] = df['series'].apply(lambda x: x[0] if x else '')
    df = df.rename(columns={'name': 'author_name'})
    df = df.dropna(subset=['title', 'description'])

    # Filtrar descripciones irrelevantes y limpiar títulos
    df = df[~df['description'].str.contains('alternate cover')]
    df['title'] = df['title'].str.replace(r"\(.*\)", "")

    # Conversión de columnas numéricas
    df['ratings_count'] = pd.to_numeric(df['ratings_count'], errors='coerce')
    df['text_reviews_count'] = pd.to_numeric(df['text_reviews_count'], errors='coerce')
    df = df.dropna(subset=['ratings_count', 'text_reviews_count'])

    # Crear una nueva característica combinada para escoger el libro más popular por work_id
    df['combined_score'] = df['ratings_count'] * 0.7 + df['text_reviews_count'] * 0.3

    return df


def select_best_books(df):
    return df.loc[df.groupby('work_id')['combined_score'].idxmax()]

In [63]:
books = preprocess_books(books)
books_authors_genres = merge_books_authors_genres(books, authors, genres)
books_authors_genres = clean_and_engineer_features(books_authors_genres)
books_authors_genres = select_best_books(books_authors_genres)

print("Número de libros únicos por work_id:", len(books_authors_genres))

/var/folders/45/_kywt6y9647bq9fvcw_fkvrw0000gn/T/ipykernel_53568/2365399967.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df['author_id'] = books_df['authors'].apply(lambda x: [author['author_id'] for author in x] if x else None)


Número de libros únicos por work_id: 19401


In [ ]:
books_authors_genres.to_csv(os.path.join(DIR, 'books_authors_genres.csv'), index=False)

Por último, creamos el dataset que contiene las features más importantes concatenadas y ponderadas por nivel de importancia por `work_id`.

In [65]:
def preprocess_text(text):
    if pd.isna(text):
        return ''
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text(text)
    return text

def select_features(df):
    df['all_features'] = (
        (df['title'].fillna('').astype(str) + ' ') +
        (df['author_name'].fillna('').astype(str) + ' ') * 2 +
        (df['genres'].fillna('').apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x)) + ' ') * 3
    ).apply(preprocess_text)
    
    return df[['work_id', 'all_features']]

data = select_features(books_authors_genres)
data.head()

,work_id,all_features
27643,62,kiffe kiffe tomorrow faiza guene faiza guene f...
9739,388,queen kat carmel st jude get life maureen mcca...
17477,969,prophet yonwood ember series jeanne duprau jea...
25152,1131,sleeping dog sonya hartnett sonya hartnett you...
24950,1512,ruby francesca lia block francesca lia block y...


## Vectorización con TF-IDF

Convertimos el texto de la columna `all_features` en vectores numéricos usando *TF-IDF*, eliminando stopwords. Luego normalizamos esos vectores para que tengan una magnitud uniforme y sean comparables.

In [66]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=100000)
tfidf_matrix = vectorizer.fit_transform(data['all_features'])
tfidf_matrix = normalize(tfidf_matrix)
print('TF-IDF matrix shape:', tfidf_matrix.shape)

TF-IDF matrix shape: (19401, 17431)


In [ ]:
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), index=data['work_id'], columns=vectorizer.get_feature_names_out())
tfidf_df.to_parquet(os.path.join(DIR, 'tf-idf.parquet'), engine='pyarrow')

## Cálculo de similitud

Calculamos una matriz de similitud entre todos los libros usando el producto escalar entre sus vectores *TF-IDF*. Así podemos medir qué tan similares son entre sí.

In [68]:
sim_matrix = linear_kernel(tfidf_matrix, tfidf_matrix, dense_output=False)
print('Similarity matrix shape:', sim_matrix.shape)

Similarity matrix shape: (19401, 19401)


In [ ]:
sim_matrix_df = pd.DataFrame(sim_matrix.toarray(), columns=data['work_id'], index=data['work_id'])
sim_matrix_df.to_parquet(os.path.join(DIR, 'sim_matrix.parquet'), engine='pyarrow')

## Ejemplo de recomendación

Definimos una función para recomendar libros similares al título indicado.

In [70]:
def recomendar_libros(data, sim_matrix_df, title, k=5):
    """
    Encuentra libros similares a un título dado y muestra información relevante.

    Parameters:
    data (pd.DataFrame): DataFrame con los libros.
    sim_matrix_df (pd.DataFrame): Matriz de similitud con índices y columnas como IDs de libros.
    title (str): Título del libro a buscar.
    k (int, optional): Número de libros similares a recomendar. Default es 5.

    Returns:
    list: Lista de diccionarios con información de los libros recomendados.
    """
    # Buscar el ID del libro
    resultado = data[data['title'].str.lower() == title.lower()]
    
    if resultado.empty:
        print(f"No se encontró un libro con el título '{title}'.")
        return []

    work_id = resultado.iloc[0]['work_id']

    if work_id not in sim_matrix_df.index:
        print(f"El libro con ID {work_id} no está en la matriz de similitud.")
        return []

    # Extraer la fila de similitudes del libro dado
    similitudes = sim_matrix_df.loc[work_id]

    # Ordenar por similitud y excluir el mismo libro
    ids_similares = similitudes.nlargest(k + 1).index[1:k+1]

    # Obtener información de los libros recomendados
    recomendaciones = []
    for similar_id in ids_similares:
        libro_similar = data[data['work_id'] == similar_id].iloc[0]
        recomendaciones.append({
            "Título": libro_similar["title"],
            "Autor": libro_similar["author_name"],
            "Géneros": libro_similar["genres"],
            "Descripción": libro_similar["description"]
        })
        print(f"Work_id: {similar_id}, Título: {libro_similar['title']}, Similitud: {similitudes[similar_id]}")

    # Mostrar información del libro consultado
    print(f"\n Libro consultado: {resultado.iloc[0]['title']}")
    print(f"   Autor: {resultado.iloc[0]['author_name']}")
    print(f"   Géneros: {resultado.iloc[0]['genres']}")
    print(f"   Descripción: {resultado.iloc[0]['description'][:300]}...")  # Limita la descripción

    # Mostrar recomendaciones
    print("\n Recomendaciones:")
    for i, rec in enumerate(recomendaciones):
        print(f"\n{i+1}. {rec['Título']}")
        print(f"   Autor: {rec['Autor']}")
        print(f"   Géneros: {rec['Géneros']}")
        print(f"   Descripción: {rec['Descripción'][:300]}...")  # Limita la descripción

    return recomendaciones

# Ejemplo de uso:
recomendaciones = recomendar_libros(books_authors_genres, sim_matrix_df, "Queen of Shadows (Throne of Glass, #4)", k=10)


Work_id: 51666806, Título: The World of Throne of Glass, Similitud: 0.9197522301166188
Work_id: 25272014, Título: Untitled (Throne of Glass, #7), Similitud: 0.9057686518622545
Work_id: 25691778, Título: The Assassin's Blade (Throne of Glass, #0.1-0.5), Similitud: 0.8608522200626919
Work_id: 11138426, Título: Throne of Glass (Throne of Glass, #1), Similitud: 0.8435855742960275
Work_id: 25880898, Título: The Assassin and the Healer (Throne of Glass, #0.2), Similitud: 0.8427287484973486
Work_id: 25128502, Título: Heir of Fire (Throne of Glass, #3), Similitud: 0.829158098119125
Work_id: 51681900, Título: Tower of Dawn (Throne of Glass, #6), Similitud: 0.8009094544358802
Work_id: 19143595, Título: The Assassin and the Empire (Throne of Glass, #0.5), Similitud: 0.7876879295136483
Work_id: 44567388, Título: Twilight of the Gods, Similitud: 0.7586399911923403
Work_id: 18814070, Título: The Assassin and the Desert (Throne of Glass, #0.3), Similitud: 0.7193237328470201

 Libro consultado: Queen 

## Evaluación

Para evaluar nuestro sistema de recomendación, cargamos el dataset `interactions` y filtramos aquellas con valoraciones positivas (`rating >= 4`). Calculamos las métricas `precision@20`, `recall@20` y `ndcg@20`recomendando libros similares a uno de los que el usuario ya valoró positivamente, y comparando esas recomendaciones con el resto de libros que también le gustaron.

In [ ]:
interactions = pd.read_csv(os.path.join(DIR, 'interactions_filtered.csv'))
interactions = interactions.merge(books_authors_genres[['book_id', 'work_id']], on='book_id', how='left')
interactions = interactions.dropna(subset=['work_id'])

Filtramos las valoraciones para quedarnos solo con aquellas positivas (`rating >= 4`). Luego, creamos un diccionario que asigna a cada usuario los libros que ha valorado positivamente. Filtramos la matriz de similitud para recomendar y evaluar solo sobre aquellos ítems que tienen interacciones, el *ground truth*.

In [ ]:
positive_ratings = interactions[interactions['rating'] >= 4]

# Crear diccionario de libros valorados positivamente
user_positive_items = defaultdict(set)
for _, row in positive_ratings.iterrows():
    user_positive_items[row['user_id']].add(row['work_id'])

# Identificar los ítems que tienen al menos una valoración positiva
valid_items = set(positive_ratings['work_id'].unique())

# Filtrar la matriz de similitud para quedarse solo con esos ítems
sim_matrix_df = sim_matrix_df.loc[list(valid_items), list(valid_items)]

Definimos las funciones para recomendar ítems y calcular las métricas. Luego, definimos una función para realizar la evaluación utilizando estas funciones.

In [74]:
# Función de recomendación de ítems similares
def recommend_similar_items(anchor_work_id, similarity_matrix, top_k=20, exclude_ids=None):
    if anchor_work_id not in similarity_matrix.index:
        return []
    
    sim_scores = similarity_matrix.loc[anchor_work_id]
    if exclude_ids:
        sim_scores = sim_scores.drop(labels=exclude_ids, errors='ignore')
        
    top_items = sim_scores.sort_values(ascending=False).head(top_k).index.tolist()
    return top_items

# Función para calcular Precision y Recall a k
def precision_recall_at_k(recommended_items, relevant_items, k=20):
    recommended_set = set(recommended_items[:k])
    relevant_set = set(relevant_items)

    true_positives = recommended_set & relevant_set

    precision = len(true_positives) / k
    recall = len(true_positives) / len(relevant_set) if relevant_set else 0
    return precision, recall

# Función para calcular nDCG@k
def ndcg_at_k(recommended_items, relevant_items, k=20):
    recommended_set = set(recommended_items[:k])
    relevant_set = set(relevant_items)

    # Calcular DCG@k
    dcg = 0
    for i, item in enumerate(recommended_items[:k]):
        if item in relevant_set:
            dcg += 1 / np.log2(i + 2)  # i+2 porque el índice es 0-based
    
    # Calcular IDCG@k (DCG ideal)
    idcg = 0
    for i in range(min(len(relevant_items), k)):
        idcg += 1 / np.log2(i + 2)
    
    # Calcular nDCG@k
    ndcg = dcg / idcg if idcg > 0 else 0
    return ndcg

# Función para evaluar el modelo
def evaluate_recommender(user_positive_items, similarity_matrix_df, top_k=20):
    """
    Evalúa un sistema de recomendación basado en similitud de ítems utilizando métricas 
    precision@k, recall@k y ndcg@k. Para cada usuario con al menos 3 ítems valorados 
    positivamente, se elige un ítem como ancla y se recomiendan ítems similares.

    Parámetros:
        user_positive_items (dict): Diccionario user_id -> set de work_id con valoraciones positivas.
        similarity_matrix_df (pd.DataFrame): Matriz de similitud entre ítems.
        top_k (int): Número de recomendaciones a considerar para la evaluación.

    Devuelve:
        pd.DataFrame: Resultados de evaluación por usuario con métricas calculadas.
    """
    results = []

    for user_id, liked_items in user_positive_items.items():
        if len(liked_items) < 3:
            continue  # Requiere al menos un libro base y otros para comparar

        liked_items = list(liked_items)
        anchor = random.choice(liked_items)  # Ítem base
        ground_truth = set(liked_items) - {anchor}

        recommended = recommend_similar_items(anchor, similarity_matrix_df, top_k=top_k, exclude_ids={anchor})
        precision, recall = precision_recall_at_k(recommended, ground_truth, k=top_k)
        ndcg = ndcg_at_k(recommended, ground_truth, k=top_k)

        results.append({
            'user_id': user_id,
            'anchor': anchor,
            f'precision@{top_k}': precision,
            f'recall@{top_k}': recall,
            f'ndcg@{top_k}': ndcg
        })

    return pd.DataFrame(results)

Llevamos a cabo la evaluación y obtenemos los resultados de las métricas calculadas.

In [75]:
evaluation = evaluate_recommender(user_positive_items, sim_matrix_df, top_k=20)

print(f"Precision@20: {evaluation['precision@20'].mean():4f}")
print(f"Recall@20: {evaluation['recall@20'].mean():4f}")
print(f"nDCG@20: {evaluation['ndcg@20'].mean():4f}")

Precision@20: 0.013253
Recall@20: 0.084531
nDCG@20: 0.058119
